# Reasoning with Sampling: Your Base Model is Smarter Than You Think

**Paper:** [arXiv:2510.14901](https://arxiv.org/abs/2510.14901)  
**Authors:** Aayush Karan and Yilun Du (Harvard)

This notebook walks through the key concepts of the "Reasoning with Sampling" paper and demonstrates the methodology on the **MATH500** dataset.

## Overview

The paper demonstrates that **base language models** can achieve reasoning capabilities comparable to fine-tuned models through **pure sampling at inference time**, without any additional training. The key innovation is using **MCMC-based power sampling** to sample from sharpened distributions using the base model's own likelihoods.

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install -q torch transformers datasets numpy scipy matplotlib pandas tqdm

In [ ]:
import torch
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from typing import List, Dict, Tuple
from dataclasses import dataclass
from tqdm.auto import tqdm
import json
import re

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

## 1. Key Concepts

### 1.1 Power Distribution

The core idea is to sample from a **power distribution** (also called tempered distribution):

$$p_\beta(y | x) \propto p(y | x)^\beta$$

where:
- $p(y | x)$ is the base model's probability of generating response $y$ given prompt $x$
- $\beta > 1$ is the temperature parameter that **sharpens** the distribution
- Higher $\beta$ concentrates probability mass on high-likelihood sequences

### 1.2 Why Not Just Low-Temperature Sampling?

Power sampling is **NOT** equivalent to low-temperature next-token sampling because:
- **Low-temperature sampling** greedily picks high-probability next tokens
- **Power sampling** accounts for future paths, favoring tokens that lead to few but high-likelihood completions
- This is exactly the behavior needed for reasoning: narrow, high-confidence chains

### 1.3 MCMC Algorithm: Metropolis-Hastings with Random Resampling

Since directly sampling from $p_\beta(y | x)$ is intractable, the paper uses **MCMC** with this procedure:

1. Start with an initial completion $y_0$ (can be from the base model)
2. For iteration $t$:
   - **Proposal**: Pick a random span in $y_t$, regenerate it with the base model → get candidate $y'$
   - **Acceptance**: Compute acceptance ratio:
     $$\alpha = \min\left(1, \frac{p(y' | x)^\beta}{p(y_t | x)^\beta}\right) = \min\left(1, \left(\frac{p(y' | x)}{p(y_t | x)}\right)^\beta\right)$$
   - Accept $y_{t+1} = y'$ with probability $\alpha$, otherwise keep $y_{t+1} = y_t$
3. After burn-in, collect samples

### 1.4 Key Advantages

- **Training-free**: No gradient updates needed
- **Dataset-free**: No curated training data required
- **Verifier-free**: Works beyond easily verifiable domains
- **Performance**: Matches or outperforms RL methods like GRPO on MATH500

## 2. Load MATH500 Dataset

The MATH dataset contains challenging mathematical problems across different subjects.

In [ ]:
from datasets import load_dataset

# Load MATH500 dataset (subset of MATH dataset)
try:
    # Try loading from HuggingFace
    dataset = load_dataset("lighteval/MATH", split="train")
    # Take first 500 for MATH500
    math500 = dataset.select(range(min(500, len(dataset))))
    print(f"Loaded {len(math500)} problems from MATH dataset")
except:
    print("Could not load MATH dataset from HuggingFace. Creating synthetic examples...")
    # Create synthetic math problems for demonstration
    math500 = [
        {
            "problem": "What is 2 + 2?",
            "solution": "2 + 2 = 4",
            "answer": "4",
            "subject": "arithmetic",
            "level": "Level 1"
        },
        {
            "problem": "Solve for x: 2x + 5 = 13",
            "solution": "2x + 5 = 13\\n2x = 8\\nx = 4",
            "answer": "4",
            "subject": "algebra",
            "level": "Level 2"
        },
        {
            "problem": "What is the derivative of x^2 + 3x?",
            "solution": "Using power rule: d/dx(x^2 + 3x) = 2x + 3",
            "answer": "2x + 3",
            "subject": "calculus",
            "level": "Level 3"
        }
    ]
    print(f"Created {len(math500)} synthetic examples")

# Display sample problem
sample_idx = 0
sample = math500[sample_idx] if isinstance(math500, list) else math500[sample_idx]
print("\n" + "="*80)
print("Sample Problem:")
print("="*80)
print(f"Problem: {sample['problem']}")
print(f"\nAnswer: {sample['answer']}")
if 'subject' in sample:
    print(f"Subject: {sample['subject']}")
if 'level' in sample:
    print(f"Level: {sample['level']}")

## 3. Implement MCMC Power Sampling

We'll implement a simplified version of the MCMC algorithm. For a real implementation, you would use a full LLM. Here we'll create a mock implementation that demonstrates the concepts.

In [ ]:
@dataclass
class SamplingResult:
    """Container for sampling results"""
    text: str
    log_prob: float
    tokens: List[str]
    token_log_probs: List[float]
    
    @property
    def prob(self) -> float:
        """Convert log probability to probability"""
        return np.exp(self.log_prob)


class MockLLM:
    """
    Mock LLM for demonstration purposes.
    In practice, you would use a real model like Qwen2.5-Math-7B.
    """
    
    def __init__(self, temperature: float = 1.0):
        self.temperature = temperature
        
    def generate(self, prompt: str, max_tokens: int = 50) -> SamplingResult:
        """
        Generate a completion with log probabilities.
        This is a mock implementation - replace with real LLM.
        """
        # Mock generation - in practice, use model.generate()
        completions = [
            "Let's solve this step by step. First, we identify the equation.",
            "To solve this problem, we need to apply the formula.",
            "The answer is obtained by calculating the value."
        ]
        
        # Random selection weighted by temperature
        idx = np.random.choice(len(completions))
        text = completions[idx]
        tokens = text.split()
        
        # Mock log probabilities (normally from model)
        token_log_probs = np.random.uniform(-2, -0.5, len(tokens)).tolist()
        total_log_prob = sum(token_log_probs)
        
        return SamplingResult(
            text=text,
            log_prob=total_log_prob,
            tokens=tokens,
            token_log_probs=token_log_probs
        )
    
    def compute_log_prob(self, prompt: str, completion: str) -> float:
        """
        Compute log probability of a completion given a prompt.
        In practice, use model.compute_log_likelihood()
        """
        # Mock implementation - in practice, run model in eval mode
        tokens = completion.split()
        # Simulate log probs based on completion quality
        base_log_prob = -len(tokens) * np.random.uniform(0.5, 1.5)
        return base_log_prob
    
    def resample_span(self, text: str, start_pos: int, end_pos: int) -> str:
        """
        Resample a span of text.
        This is the proposal mechanism in MCMC.
        """
        tokens = text.split()
        prefix = " ".join(tokens[:start_pos])
        suffix = " ".join(tokens[end_pos:])
        
        # Generate new middle part
        new_span_options = [
            "we can determine that",
            "it follows that",
            "we observe that",
            "this gives us"
        ]
        new_span = np.random.choice(new_span_options)
        
        return f"{prefix} {new_span} {suffix}".strip()


class PowerSampler:
    """
    Implements MCMC Power Sampling for LLM reasoning.
    """
    
    def __init__(self, model: MockLLM, beta: float = 2.0):
        """
        Args:
            model: Language model to use
            beta: Power parameter (beta > 1 sharpens distribution)
        """
        self.model = model
        self.beta = beta
        self.acceptance_history = []
        
    def acceptance_probability(self, log_prob_current: float, log_prob_proposal: float) -> float:
        """
        Compute Metropolis-Hastings acceptance probability.
        
        alpha = min(1, (p(y'|x) / p(y|x))^beta)
        In log space: log(alpha) = min(0, beta * (log p(y'|x) - log p(y|x)))
        """
        log_ratio = self.beta * (log_prob_proposal - log_prob_current)
        return min(1.0, np.exp(log_ratio))
    
    def sample(self, prompt: str, n_iterations: int = 100, burn_in: int = 20) -> Tuple[List[SamplingResult], List[float]]:
        """
        Run MCMC power sampling.
        
        Args:
            prompt: Input prompt
            n_iterations: Number of MCMC iterations
            burn_in: Number of initial samples to discard
            
        Returns:
            samples: List of samples after burn-in
            acceptance_rates: Acceptance rate at each iteration
        """
        # Initialize with base model sample
        current = self.model.generate(prompt)
        current_log_prob = self.model.compute_log_prob(prompt, current.text)
        
        samples = []
        acceptance_rates = []
        n_accepted = 0
        
        for i in tqdm(range(n_iterations), desc="MCMC Sampling"):
            # Proposal: resample random span
            tokens = current.text.split()
            if len(tokens) > 3:
                # Pick random span (at least 2 tokens)
                span_start = np.random.randint(0, len(tokens) - 2)
                span_end = np.random.randint(span_start + 1, len(tokens))
                proposal_text = self.model.resample_span(current.text, span_start, span_end)
            else:
                # If too short, generate new sample
                proposal = self.model.generate(prompt)
                proposal_text = proposal.text
            
            # Compute proposal log probability
            proposal_log_prob = self.model.compute_log_prob(prompt, proposal_text)
            
            # Acceptance step
            alpha = self.acceptance_probability(current_log_prob, proposal_log_prob)
            
            if np.random.random() < alpha:
                # Accept proposal
                current_text = proposal_text
                current_log_prob = proposal_log_prob
                n_accepted += 1
            # else: keep current (rejection)
            
            # Track acceptance rate
            acceptance_rates.append(n_accepted / (i + 1))
            
            # Collect sample after burn-in
            if i >= burn_in:
                sample = SamplingResult(
                    text=current_text,
                    log_prob=current_log_prob,
                    tokens=current_text.split(),
                    token_log_probs=[current_log_prob / len(current_text.split())] * len(current_text.split())
                )
                samples.append(sample)
        
        self.acceptance_history = acceptance_rates
        return samples, acceptance_rates


print("✓ MCMC Power Sampling implementation complete")

## 4. Demonstration: Standard Sampling vs MCMC Power Sampling

Let's compare standard sampling with MCMC power sampling on a math problem.

In [ ]:
# Create mock LLM
model = MockLLM(temperature=1.0)

# Get a sample problem
problem = math500[0] if isinstance(math500, list) else math500[0]
prompt = f"Problem: {problem['problem']}\n\nSolution:"

print("Problem:")
print(problem['problem'])
print("\n" + "="*80)

# Standard sampling
print("\n[1] STANDARD SAMPLING (Multiple independent samples)")
print("="*80)
standard_samples = []
for i in range(10):
    sample = model.generate(prompt)
    log_prob = model.compute_log_prob(prompt, sample.text)
    standard_samples.append(SamplingResult(
        text=sample.text,
        log_prob=log_prob,
        tokens=sample.tokens,
        token_log_probs=sample.token_log_probs
    ))

print(f"Generated {len(standard_samples)} independent samples")
print(f"Average log-prob: {np.mean([s.log_prob for s in standard_samples]):.3f}")
print(f"Std log-prob: {np.std([s.log_prob for s in standard_samples]):.3f}")

# MCMC Power sampling
print("\n[2] MCMC POWER SAMPLING (β=2.0)")
print("="*80)
power_sampler = PowerSampler(model, beta=2.0)
mcmc_samples, acceptance_rates = power_sampler.sample(
    prompt, 
    n_iterations=100, 
    burn_in=20
)

print(f"Generated {len(mcmc_samples)} MCMC samples (after burn-in)")
print(f"Average log-prob: {np.mean([s.log_prob for s in mcmc_samples]):.3f}")
print(f"Std log-prob: {np.std([s.log_prob for s in mcmc_samples]):.3f}")
print(f"Final acceptance rate: {acceptance_rates[-1]:.2%}")

## 5. Visualize MCMC Convergence

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Log probability distribution comparison
ax = axes[0, 0]
standard_log_probs = [s.log_prob for s in standard_samples]
mcmc_log_probs = [s.log_prob for s in mcmc_samples]

ax.hist(standard_log_probs, bins=15, alpha=0.6, label='Standard Sampling', color='blue')
ax.hist(mcmc_log_probs, bins=15, alpha=0.6, label='MCMC Power Sampling (β=2.0)', color='red')
ax.axvline(np.mean(standard_log_probs), color='blue', linestyle='--', linewidth=2, label='Standard Mean')
ax.axvline(np.mean(mcmc_log_probs), color='red', linestyle='--', linewidth=2, label='MCMC Mean')
ax.set_xlabel('Log Probability')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Log Probabilities')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Acceptance rate over iterations
ax = axes[0, 1]
ax.plot(acceptance_rates, linewidth=2, color='green')
ax.axhline(acceptance_rates[-1], color='red', linestyle='--', 
           label=f'Final: {acceptance_rates[-1]:.2%}')
ax.axvline(20, color='gray', linestyle=':', label='Burn-in end')
ax.set_xlabel('Iteration')
ax.set_ylabel('Acceptance Rate')
ax.set_title('MCMC Acceptance Rate Over Time')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Log prob trace (MCMC chain)
ax = axes[1, 0]
ax.plot(range(len(mcmc_samples)), mcmc_log_probs, linewidth=1, alpha=0.7, color='purple')
ax.axhline(np.mean(mcmc_log_probs), color='red', linestyle='--', 
           label=f'Mean: {np.mean(mcmc_log_probs):.2f}')
ax.set_xlabel('Sample Index (post burn-in)')
ax.set_ylabel('Log Probability')
ax.set_title('MCMC Chain Trace Plot')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Box plot comparison
ax = axes[1, 1]
data_to_plot = [standard_log_probs, mcmc_log_probs]
bp = ax.boxplot(data_to_plot, labels=['Standard', 'MCMC (β=2.0)'], 
                patch_artist=True, widths=0.6)
colors = ['lightblue', 'lightcoral']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
ax.set_ylabel('Log Probability')
ax.set_title('Log Probability Comparison')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('mcmc_sampling_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved as 'mcmc_sampling_analysis.png'")

## 6. Effect of Beta Parameter

Let's explore how different values of β affect the sampling.

In [ ]:
# Test different beta values
beta_values = [1.0, 1.5, 2.0, 3.0, 5.0]
results_by_beta = {}

print("Testing different β values...")
print("="*80)

for beta in beta_values:
    sampler = PowerSampler(model, beta=beta)
    samples, acc_rates = sampler.sample(prompt, n_iterations=100, burn_in=20)
    
    results_by_beta[beta] = {
        'samples': samples,
        'log_probs': [s.log_prob for s in samples],
        'acceptance_rate': acc_rates[-1]
    }
    
    print(f"β={beta:.1f}: Mean log-prob={np.mean(results_by_beta[beta]['log_probs']):.3f}, "
          f"Acceptance rate={acc_rates[-1]:.2%}")

# Visualize effect of beta
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Mean log-prob vs beta
ax = axes[0]
means = [np.mean(results_by_beta[b]['log_probs']) for b in beta_values]
stds = [np.std(results_by_beta[b]['log_probs']) for b in beta_values]
ax.errorbar(beta_values, means, yerr=stds, marker='o', markersize=8, 
            linewidth=2, capsize=5, capthick=2)
ax.set_xlabel('β (Power Parameter)', fontsize=12)
ax.set_ylabel('Mean Log Probability', fontsize=12)
ax.set_title('Effect of β on Sample Quality', fontsize=14)
ax.grid(True, alpha=0.3)

# Plot 2: Acceptance rate vs beta
ax = axes[1]
acc_rates = [results_by_beta[b]['acceptance_rate'] for b in beta_values]
ax.plot(beta_values, acc_rates, marker='s', markersize=8, linewidth=2, color='green')
ax.set_xlabel('β (Power Parameter)', fontsize=12)
ax.set_ylabel('Acceptance Rate', fontsize=12)
ax.set_title('Effect of β on MCMC Acceptance', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig('beta_parameter_effect.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Beta parameter analysis saved as 'beta_parameter_effect.png'")

## 7. Wilcoxon Signed-Rank Test: MCMC vs Standard Sampling

Now we apply the **Wilcoxon signed-rank test** to statistically compare log-probabilities between MCMC power sampling and standard sampling.

### What is the Wilcoxon Signed-Rank Test?

- **Non-parametric test** for paired samples (doesn't assume normal distribution)
- Tests whether the median difference between paired observations is zero
- **Null hypothesis** (H₀): The two sampling methods produce the same log-probability distribution
- **Alternative hypothesis** (H₁): MCMC produces different (typically higher) log-probabilities

### Why Use It Here?

- We want to test if MCMC power sampling **significantly** improves log-probabilities
- The test is robust to outliers and doesn't assume normality
- Perfect for comparing two sampling strategies on the same problems

In [ ]:
# Run comparison on multiple problems
n_problems = min(20, len(math500))  # Test on 20 problems
n_samples_per_method = 10  # 10 samples per method per problem

print(f"Running comparison on {n_problems} problems...")
print("="*80)

comparison_data = []

for i in tqdm(range(n_problems), desc="Processing problems"):
    problem = math500[i] if isinstance(math500, list) else math500[i]
    prompt = f"Problem: {problem['problem']}\n\nSolution:"
    
    # Standard sampling
    standard_log_probs = []
    for _ in range(n_samples_per_method):
        sample = model.generate(prompt)
        log_prob = model.compute_log_prob(prompt, sample.text)
        standard_log_probs.append(log_prob)
    
    # MCMC power sampling (β=2.0)
    sampler = PowerSampler(model, beta=2.0)
    mcmc_samples_result, _ = sampler.sample(prompt, n_iterations=50, burn_in=10)
    # Take first n_samples_per_method
    mcmc_log_probs = [s.log_prob for s in mcmc_samples_result[:n_samples_per_method]]
    
    # Store results
    comparison_data.append({
        'problem_id': i,
        'problem': problem['problem'][:50] + '...',  # truncate for display
        'standard_mean': np.mean(standard_log_probs),
        'mcmc_mean': np.mean(mcmc_log_probs),
        'standard_max': np.max(standard_log_probs),
        'mcmc_max': np.max(mcmc_log_probs),
        'standard_log_probs': standard_log_probs,
        'mcmc_log_probs': mcmc_log_probs
    })

# Create DataFrame for analysis
df_comparison = pd.DataFrame([
    {
        'problem_id': d['problem_id'],
        'problem': d['problem'],
        'standard_mean': d['standard_mean'],
        'mcmc_mean': d['mcmc_mean'],
        'difference': d['mcmc_mean'] - d['standard_mean']
    }
    for d in comparison_data
])

print("\nSummary Statistics:")
print(df_comparison[['standard_mean', 'mcmc_mean', 'difference']].describe())
print(f"\nProblems where MCMC > Standard: {(df_comparison['difference'] > 0).sum()} / {n_problems}")

In [ ]:
# Perform Wilcoxon Signed-Rank Test
print("\n" + "="*80)
print("WILCOXON SIGNED-RANK TEST")
print("="*80)

# Extract paired samples (mean log-prob per problem)
standard_means = df_comparison['standard_mean'].values
mcmc_means = df_comparison['mcmc_mean'].values

# Perform the test
statistic, p_value = stats.wilcoxon(standard_means, mcmc_means, alternative='less')
# 'less' means we test if standard < mcmc (i.e., mcmc is better)

print(f"\nNull Hypothesis (H₀): MCMC and Standard sampling produce the same log-probabilities")
print(f"Alternative Hypothesis (H₁): MCMC produces higher log-probabilities")
print(f"\nTest Results:")
print(f"  Wilcoxon statistic: {statistic:.4f}")
print(f"  p-value: {p_value:.6f}")
print(f"  Significance level (α): 0.05")

if p_value < 0.05:
    print(f"\n✓ RESULT: REJECT H₀ (p={p_value:.6f} < 0.05)")
    print(f"  → MCMC power sampling produces SIGNIFICANTLY higher log-probabilities!")
else:
    print(f"\n✗ RESULT: FAIL TO REJECT H₀ (p={p_value:.6f} ≥ 0.05)")
    print(f"  → No significant difference detected")

# Effect size (median of differences)
median_diff = np.median(df_comparison['difference'])
mean_diff = np.mean(df_comparison['difference'])
print(f"\nEffect Size:")
print(f"  Median difference: {median_diff:.4f}")
print(f"  Mean difference: {mean_diff:.4f}")
print(f"  Mean improvement: {(mean_diff / np.abs(np.mean(standard_means)) * 100):.2f}%")

## 8. Visualize Wilcoxon Test Results

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Paired comparison
ax = axes[0, 0]
x = np.arange(len(df_comparison))
width = 0.35
ax.bar(x - width/2, df_comparison['standard_mean'], width, label='Standard', alpha=0.8, color='blue')
ax.bar(x + width/2, df_comparison['mcmc_mean'], width, label='MCMC (β=2.0)', alpha=0.8, color='red')
ax.set_xlabel('Problem ID')
ax.set_ylabel('Mean Log Probability')
ax.set_title('Paired Comparison: Standard vs MCMC')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 2: Difference distribution
ax = axes[0, 1]
differences = df_comparison['difference'].values
ax.hist(differences, bins=15, alpha=0.7, color='green', edgecolor='black')
ax.axvline(0, color='red', linestyle='--', linewidth=2, label='No difference')
ax.axvline(np.median(differences), color='blue', linestyle='--', linewidth=2, 
           label=f'Median: {np.median(differences):.3f}')
ax.set_xlabel('Difference (MCMC - Standard)')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Log-Prob Differences')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Scatter plot with regression
ax = axes[1, 0]
ax.scatter(df_comparison['standard_mean'], df_comparison['mcmc_mean'], 
           alpha=0.6, s=100, color='purple', edgecolors='black', linewidth=1)
# Add diagonal line (y=x)
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),
    np.max([ax.get_xlim(), ax.get_ylim()]),
]
ax.plot(lims, lims, 'k--', alpha=0.5, linewidth=2, label='y=x (no difference)')
ax.set_xlabel('Standard Sampling (Mean Log-Prob)')
ax.set_ylabel('MCMC Sampling (Mean Log-Prob)')
ax.set_title('Standard vs MCMC: Scatter Plot')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Statistical summary
ax = axes[1, 1]
ax.axis('off')

# Create text summary
summary_text = f"""
WILCOXON SIGNED-RANK TEST RESULTS
{'='*50}

Sample Size: {len(df_comparison)} problems
Samples per method: {n_samples_per_method}

Test Statistics:
  • Wilcoxon statistic: {statistic:.4f}
  • p-value: {p_value:.6f}
  • Significance level: α = 0.05

Effect Size:
  • Mean difference: {mean_diff:.4f}
  • Median difference: {median_diff:.4f}
  • Std of differences: {np.std(differences):.4f}

Conclusion:
"""

if p_value < 0.05:
    summary_text += f"  ✓ MCMC is SIGNIFICANTLY better\n"
    summary_text += f"    (p = {p_value:.6f} < 0.05)\n\n"
    summary_text += f"  MCMC power sampling achieves\n"
    summary_text += f"  {(mean_diff / np.abs(np.mean(standard_means)) * 100):.2f}% improvement\n"
    summary_text += f"  in log-probability"
else:
    summary_text += f"  ✗ No significant difference\n"
    summary_text += f"    (p = {p_value:.6f} ≥ 0.05)"

ax.text(0.05, 0.95, summary_text, transform=ax.transAxes, 
        fontsize=11, verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('wilcoxon_test_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Wilcoxon test visualization saved as 'wilcoxon_test_results.png'")

## 9. Additional Analysis: Pass@k Performance

One key metric in the paper is **Pass@k** - the probability that at least one of k samples is correct.

For MCMC power sampling, we expect:
- Higher quality samples overall (higher log-probs)
- Better Pass@k performance (more correct answers in top-k)

In [ ]:
def estimate_pass_at_k(n_correct: int, n_total: int, k: int) -> float:
    """
    Estimate Pass@k using the formula from the paper.
    
    Pass@k = E[1 - C(n_total - n_correct, k) / C(n_total, k)]
    
    where C(n, k) is the binomial coefficient "n choose k"
    """
    if n_total < k:
        return 0.0
    if n_correct >= k:
        return 1.0
    
    from scipy.special import comb
    return 1.0 - (comb(n_total - n_correct, k) / comb(n_total, k))


# Simulate Pass@k analysis
print("Pass@k Analysis (Simulated)")
print("="*80)
print("\nNote: In practice, you would evaluate actual correctness.")
print("Here we simulate by assuming higher log-prob → higher chance of correctness\n")

k_values = [1, 5, 10, 20, 50]
n_total = 100  # Total samples

# Simulate: assume top 30% of standard and top 50% of MCMC are correct
# (MCMC produces better samples)
standard_n_correct = int(0.30 * n_total)
mcmc_n_correct = int(0.50 * n_total)

print(f"Simulated scenario:")
print(f"  Standard sampling: {standard_n_correct}/{n_total} correct ({100*standard_n_correct/n_total:.0f}%)")
print(f"  MCMC sampling: {mcmc_n_correct}/{n_total} correct ({100*mcmc_n_correct/n_total:.0f}%)\n")

results_passk = []
for k in k_values:
    standard_passk = estimate_pass_at_k(standard_n_correct, n_total, k)
    mcmc_passk = estimate_pass_at_k(mcmc_n_correct, n_total, k)
    results_passk.append({
        'k': k,
        'standard': standard_passk,
        'mcmc': mcmc_passk,
        'improvement': mcmc_passk - standard_passk
    })
    print(f"Pass@{k:2d}:  Standard={standard_passk:.3f}  |  MCMC={mcmc_passk:.3f}  |  "
          f"Δ={mcmc_passk - standard_passk:+.3f}")

# Visualize Pass@k
df_passk = pd.DataFrame(results_passk)

plt.figure(figsize=(10, 6))
plt.plot(df_passk['k'], df_passk['standard'], marker='o', markersize=8, 
         linewidth=2, label='Standard Sampling', color='blue')
plt.plot(df_passk['k'], df_passk['mcmc'], marker='s', markersize=8, 
         linewidth=2, label='MCMC Power Sampling (β=2.0)', color='red')
plt.xlabel('k (number of samples)', fontsize=12)
plt.ylabel('Pass@k', fontsize=12)
plt.title('Pass@k Performance Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
plt.ylim([0, 1.05])
plt.tight_layout()
plt.savefig('passk_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Pass@k analysis saved as 'passk_analysis.png'")

## 10. Summary and Key Takeaways

### Main Findings from the Paper

1. **Power Sampling Works**: MCMC-based power sampling can extract better reasoning from base models without fine-tuning

2. **Matches RL Performance**: On MATH500, power sampling achieves results comparable to or better than GRPO (Group Relative Policy Optimization)

3. **Training-Free**: No need for:
   - Curated datasets
   - Reward models or verifiers
   - Gradient-based optimization
   - Hyperparameter tuning for training stability

4. **Generalizes Well**: Works across:
   - Multiple model families (Qwen, Phi, etc.)
   - Different domains (math, code, science, general reasoning)

### Implementation Insights

1. **β Parameter**: 
   - β = 1: Standard sampling (no sharpening)
   - β > 1: Power sampling (sharpens toward high-likelihood)
   - Typical values: β ∈ [2, 5]
   - Trade-off: Higher β → better samples but lower acceptance rate

2. **MCMC Design**:
   - Random span resampling as proposal mechanism
   - Metropolis-Hastings acceptance based on likelihood ratio
   - Requires burn-in period (typically 20-50 iterations)

3. **Computational Cost**:
   - More expensive than standard sampling (multiple model evaluations)
   - But cheaper than training with RL
   - Can be parallelized across problems

### When to Use Power Sampling

✓ **Use when:**
- You have a base model but no training data/compute for fine-tuning
- You need better reasoning on complex tasks (math, code, logic)
- You can afford extra inference-time computation
- You want diverse high-quality samples (Pass@k scenarios)

✗ **Don't use when:**
- Simple tasks where standard sampling suffices
- Extreme latency constraints
- You already have a well-tuned RL model

### Next Steps

To use this in practice:
1. Clone the repository: `https://github.com/aakaran/reasoning-with-sampling`
2. Install dependencies and download a base model (e.g., Qwen2.5-Math-7B)
3. Run power sampling with different β values
4. Evaluate with your domain-specific metrics
5. Compare against your baseline (fine-tuned model or standard sampling)

**Paper citation:**
```
Karan, A., & Du, Y. (2024). Reasoning with Sampling: Your Base Model is 
Smarter Than You Think. arXiv preprint arXiv:2510.14901.
```

## Appendix: Export Results for Further Analysis

In [ ]:
# Save comparison results to CSV
df_comparison.to_csv('mcmc_vs_standard_comparison.csv', index=False)
print("✓ Comparison data saved to 'mcmc_vs_standard_comparison.csv'")

# Save Pass@k results
df_passk.to_csv('passk_results.csv', index=False)
print("✓ Pass@k results saved to 'passk_results.csv'")

# Save statistical test results
test_results = {
    'test': 'Wilcoxon Signed-Rank Test',
    'statistic': statistic,
    'p_value': p_value,
    'alpha': 0.05,
    'significant': p_value < 0.05,
    'mean_difference': mean_diff,
    'median_difference': median_diff,
    'n_problems': len(df_comparison),
    'n_samples_per_method': n_samples_per_method
}

with open('wilcoxon_test_results.json', 'w') as f:
    json.dump(test_results, f, indent=2)
print("✓ Statistical test results saved to 'wilcoxon_test_results.json'")

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. mcmc_sampling_analysis.png - MCMC convergence visualization")
print("  2. beta_parameter_effect.png - Effect of β parameter")
print("  3. wilcoxon_test_results.png - Statistical comparison")
print("  4. passk_analysis.png - Pass@k performance")
print("  5. mcmc_vs_standard_comparison.csv - Raw comparison data")
print("  6. passk_results.csv - Pass@k data")
print("  7. wilcoxon_test_results.json - Statistical test summary")